In [ ]:
from automatic_prompt_engineer import ape

words = ["sane", "direct", "informally", "unpopular", "subtractive", "nonresidential",
         "inexact", "uptown", "incomparable", "powerful", "gaseous", "evenly", "formality",
         "deliberately", "off"]
antonyms = ["insane", "indirect", "formally", "popular", "additive", "residential",
            "exact", "downtown", "comparable", "powerless", "solid", "unevenly", "informality",
            "accidentally", "on"]
            
eval_template = \
"""Instruction: [PROMPT]
Input: [INPUT]
Output: [OUTPUT]"""

result, demo_fn = ape.simple_ape(
    dataset=(words, antonyms),
    eval_template=eval_template,
)

In [1]:
import os

In [ ]:
os.chdir("../")

In [2]:
os.getcwd()

'/teamspace/studios/this_studio/Precision-RAG-For_Enterprise-Grade-RAG-Systems'

In [5]:
!pip install langchain

In [8]:
%pip install --upgrade --quiet  \
    langchain-pinecone \
    langchain-openai \
    langchain \
    langchain-community \
    pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [2]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores.pinecone import PineconeStore
# from langchain.document_loaders.fs.pdf import PDFLoader
# from langchain.document_loaders.fs.directory import DirectoryLoader
# from langchain.utils.pinecone_client import pinecone

from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# import os
# from langchain_pinecone import PineconeVectorStore
# from langchain_openai import OpenAIEmbeddings


In [ ]:

PINECONE_INDEX_NAME = "pdf-doc"
PINECONE_NAME_SPACE = "pdf-namespace"
# Name of directory to retrieve your files from
# Make sure to add your PDF files inside the 'docs' folder
file_path = 'data'

def run():
    try:
        # load raw docs from all files in the directory
        directory_loader = PDFLoader("../data/10 Academy Cohort B - Weekly Challenge_ Week - 7.pdf")

        raw_docs = directory_loader.load()

        # Split text into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

        docs = text_splitter.split_documents(raw_docs)
        print('split docs', docs)

        print('creating vector store...')
        # create and store the embeddings in the vector store
        embeddings = OpenAIEmbeddings()
        index = pinecone.Index(PINECONE_INDEX_NAME)  # change to your own index name

        # embed the PDF documents
        PineconeStore.from_documents(docs, embeddings, {
            'pineconeIndex': index,
            'namespace': PINECONE_NAME_SPACE,
            'textKey': 'text',
        })
    except Exception as error:
        print('error', error)
        raise Exception('Failed to ingest your data')